In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

release = 'https://chromedriver.storage.googleapis.com/LATEST_RELEASE'
version = requests.get(release).text
service = Service(ChromeDriverManager(version=version).install())
d = webdriver.Chrome(service=service)
try:
    d.get("https://cafe.naver.com/joonggonara")
    elem = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#topLayerQueryInput")))
    elem.send_keys("자전거")
    elem.send_keys(Keys.RETURN)
    
    iframe = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#cafe_main")))
    d.switch_to.frame(iframe)
    
    article = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".article")))
    print(article.text)
except Exception as e:
    print(e)
finally:
    d.close()
    d.quit()

WebDriverException: Message: Service /Users/syshin/.wdm/drivers/chromedriver/mac_arm64/114.0.5735.90/chromedriver unexpectedly exited. Status code was: -9


In [ ]:
# 중고나라에서 자전거 검색하여 1~5페이지 제목, 작성자, 작성일 excel에 표시

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from openpyxl import Workbook
from selenium import webdriver


def crawl(*args):
    # Excel 연결
    wb = Workbook()

    # 중고나라 접속
    release = 'https://chromedriver.storage.googleapis.com/LATEST_RELEASE'
    version = requests.get(release).text
    service = Service(ChromeDriverManager(version=version).install())
    d = webdriver.Chrome(service=service)
    try:
        for arg in args:
            ws = wb.create_sheet(arg)
            # ws.append([arg])
            ws.append(['title', 'writer', 'date'])

            d.get("https://cafe.naver.com/joonggonara")
            elem = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#topLayerQueryInput")))
            elem.send_keys(arg)
            elem.send_keys(Keys.RETURN)

            iframe = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#cafe_main")))
            d.switch_to.frame(iframe)
            for i in range(2, 6):
                trs = d.find_elements(By.CSS_SELECTOR, '#main-area > div:nth-child(5) > table > tbody > tr')
                for tr in trs:
                    title = tr.find_element(By.CSS_SELECTOR, 'td.td_article > div.board-list > div > a').text
                    writer = tr.find_element(By.CSS_SELECTOR, 'td.td_name > div > table > tbody > tr > td > a').text
                    date = tr.find_element(By.CSS_SELECTOR, 'td.td_date').text
                    ws.append([title, writer, date])
                if i==6:
                    break
                page_area = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#main-area > div.prev-next")))
                go_to_page = page_area.find_element(By.LINK_TEXT, str(i))
                go_to_page.click()
            d.switch_to.default_content()
    except Exception as e:
        print(e)
    finally:
        d.close()
        d.quit()
        wb.save("joonggonara.xlsx")

In [36]:
crawl("자전거", "노트북", "키보드", "모니터")

WebDriverException: Message: Service /Users/syshin/.wdm/drivers/chromedriver/mac_arm64/114.0.5735.90/chromedriver unexpectedly exited. Status code was: -9


In [37]:
!which chromedriver

/opt/homebrew/bin/chromedriver
